In [ ]:
import asyncio
from twikit import Client, TooManyRequests
from configparser import ConfigParser
import pandas as pd
from datetime import datetime, timedelta

# Constantes
MINIMUMS = 100
REQUETE = 'ballon d\'or '  
# Lire les informations de connexion depuis le fichier config.ini
config = ConfigParser()
config.read('config.ini')
username = config['X']['username']
email = config['X']['email']
password = config['X']['password']

client = Client(language='fr-FR')  # Configurer pour obtenir des résultats en français

# Fonction principale async
async def main():
    # Authentification
    await client.login(auth_info_1=username, auth_info_2=email, password=password)
    
    # Sauvegarder les cookies après l'authentification
    client.save_cookies('cookies.json')
    client.load_cookies('cookies.json')  # Charger les cookies pour les sessions futures

    # Initialisation de la liste pour les tweets et compteur
    donne = []
    tweet_count = 0

    # Rechercher les tweets avec la requête spécifiée
    tweets = await client.search_tweet(REQUETE, product='Top')

    # Boucle pour paginer les tweets
    while tweet_count < MINIMUMS:
            # Récupérer la page suivante de tweets
            encore = await tweets.next()

            for tweet in encore:
                post_data = {
                    'Texte': tweet.full_text,
                    'Nombre_Likes': tweet.favorite_count,
                    'Nombre_Retweets': tweet.retweet_count,
                    'Followers_Utilisateur': tweet.user.followers_count,
                    'Utilisateur': tweet.user.screen_name,
                    'Date_Publication': tweet.created_at
                }
                donne.append(post_data)
                tweet_count += 1

            
                if tweet_count >= MINIMUMS:
                    break
           
        

    if donne:
        df = pd.DataFrame(donne)
        df.to_csv('tweets_data.csv', index=False)
        print(f"{len(df)} tweets collectés et sauvegardés dans 'tweets_data.csv'")
    else:
        print("Aucune donnée collectée.")

# Exécuter la fonction async
await main()




100 tweets collectés et sauvegardés dans 'tweets_data.csv'


In [ ]:

donne = pd.read_csv('tweets_data.csv')
donne.head()

#appliquer des nettoyages
def netoyer(text):
    text = text.lower()
    return ' '.join(mot for mot in text.split() if mot.isalpha())

#appliquer le netooyage
donne['Texte'] = donne['Texte'].apply(netoyer)




In [20]:
from transformers import pipeline

# Créer un pipeline d'analyse de sentiment en utilisant DistilBERT
sentiment = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
# Analyser un texte
for texte in donne['Texte'] :
    resultat = sentiment(texte)
    #ajouter la colonne sentiment a notre dataframe
    donne['sentiment'] = resultat[0]['label']
    donne['score'] = resultat[0]['score']
    
donne.head(10)

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


,Texte,Nombre_Likes,Nombre_Retweets,Followers_Utilisateur,Utilisateur,Date_Publication,sentiment,score
0,flash apple annonce que sera vendu sans câble ...,1986,55,169734,Pediavenir,Thu Sep 19 17:36:08 +0000 2024,NEGATIVE,0.905765
1,apple dévoile qui sera disponible dès le avec ...,210,33,163248,viewsfrance,Mon Sep 09 19:10:30 +0000 2024,NEGATIVE,0.905765
2,une pense à toutes ces personnes qui vont comm...,255,16,429,_LuCaS_tS_,Sun Sep 08 07:56:44 +0000 2024,NEGATIVE,0.905765
3,est enfin sortie et cette année apple nous pro...,675,57,9105,Jeff_Mbogne,Thu Sep 26 20:24:00 +0000 2024,NEGATIVE,0.905765
4,iphone déjà dispo sur temu pour seulement ai p...,212,10,35302,JohnnyBaudelai2,Thu Sep 12 18:12:11 +0000 2024,NEGATIVE,0.905765
5,nouvel iphone dispo en couleurs ecran plus rés...,1318,153,89351,StevenLathoud,Mon Sep 09 18:10:10 +0000 2024,NEGATIVE,0.905765
6,cela fait moins de a présenté il de la plus gr...,276,22,20740,EnzoManis,Tue Sep 10 16:02:21 +0000 2024,NEGATIVE,0.905765
7,apple vient de lancer il de la plus grande mis...,12679,648,35592,Jouhatsu_ai,Tue Sep 10 13:00:06 +0000 2024,NEGATIVE,0.905765
8,enfin révélé une fuite dévoile tous les détail...,226,20,64297,AlexNoubeyou,Sun Aug 25 09:01:06 +0000 2024,NEGATIVE,0.905765
9,apple vient de lancer les nouveaux iphone et i...,329,17,20740,EnzoManis,Mon Sep 16 12:32:13 +0000 2024,NEGATIVE,0.905765
